In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px

In [ ]:
df= pd.read_csv('../data/analysis/biomarker_measurement.csv')
df.head()


,person_id,index_visit_id,index_start_date,index_end_date,procedure_concept_id,procedure_datetime,measurement_concept_id,measurement_datetime,value_as_number,range_low,range_high,los_days,plos_days,phase_label
0,942,1204,2023-03-11,2023-04-08,4203153,2023-04-06 00:00:00,42868738,2023-02-20 13:43:37.342,0.8,0.5,1.6,28,2,Pre-Op
1,942,1204,2023-03-11,2023-04-08,4203153,2023-04-06 00:00:00,3034022,2023-02-20 13:43:37.389,932.0,85.0,450.0,28,2,Pre-Op
2,942,1204,2023-03-11,2023-04-08,4203153,2023-04-06 00:00:00,3049746,2023-02-20 13:43:37.42,73.0,NaN,NaN,28,2,Pre-Op
3,942,1204,2023-03-11,2023-04-08,4203153,2023-04-06 00:00:00,40765586,2023-02-20 14:02:28.238,6.3,3.1,18.5,28,2,Pre-Op
4,942,1204,2023-03-11,2023-04-08,4203153,2023-04-06 00:00:00,3034426,2023-02-20 14:02:30.801,13.1,NaN,NaN,28,2,Pre-Op


In [3]:
df.describe()

,person_id,index_visit_id,procedure_concept_id,measurement_concept_id,value_as_number,range_low,range_high,los_days,plos_days
count,24326.000000,24326.000000,2.432600e+04,2.432600e+04,24326.000000,16322.000000,16322.000000,24326.000000,24326.000000
mean,1497.139234,4673.945573,4.199862e+06,1.882149e+07,167.863624,41.589505,118.827307,20.529516,7.034736
std,326.835595,2676.827723,1.001381e+05,1.932803e+07,644.159564,59.544913,170.215306,17.799336,8.712273
min,942.000000,18.000000,4.012932e+06,3.027193e+06,0.000000,0.500000,1.500000,1.000000,0.000000
25%,1213.000000,2459.250000,4.142628e+06,3.034426e+06,1.200000,0.500000,1.600000,10.000000,2.000000
50%,1502.000000,4623.000000,4.203153e+06,3.049746e+06,13.100000,3.100000,18.500000,15.000000,3.000000
75%,1772.000000,7026.000000,4.284104e+06,4.286874e+07,98.000000,70.000000,140.000000,25.000000,9.000000
max,2088.000000,9307.000000,4.304688e+06,4.286874e+07,26876.000000,207.000000,450.000000,147.000000,61.000000


In [19]:
# 1. Convert to datetime using 'mixed' to handle varying precision (with/without milliseconds)
df['measurement_datetime'] = pd.to_datetime(df['measurement_datetime'], format='mixed')
df['procedure_datetime'] = pd.to_datetime(df['procedure_datetime'], format='mixed')

# 2. Calculate 'Days from Procedure' (Your X-axis)
# This creates a timeline where 0 is the moment of surgery
df['days_from_proc'] = (df['measurement_datetime'] - df['procedure_datetime']).dt.total_seconds() / 86400

# 3. Ensure your color variable is numeric for the continuous scale
df['plos_days'] = pd.to_numeric(df['plos_days'], errors='coerce')

In [ ]:
example_concept = 42868738 
plot_df = df[df['measurement_concept_id'] == example_concept].copy()

In [24]:
fig = px.line(
    plot_df.sort_values(by=['index_visit_id', 'days_from_proc']),
    x="days_from_proc", 
    y="value_as_number",
    color="plos_days",                # CONTINUOUS SCALE (Reviewer requirement)
    line_group="index_visit_id",      # UNIQUE LINE PER VISIT
    markers=True,                     # DISTINCT MEASUREMENT POINTS
    color_continuous_scale="Blues_r", # DARK BLUE (Short LOS) to LIGHT BLUE (Long LOS)
    hover_data=["person_id", "index_visit_id", "phase_label"],
    title=f"Trend for Measurement {example_concept} by Post-Op LOS"
)

TypeError: line() got an unexpected keyword argument 'color_continuous_scale'

In [ ]:
# Add a vertical line at 0 to show exactly when the procedure happened
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Procedure")

fig.update_layout(
    xaxis_title="Days Relative to Procedure",
    yaxis_title="Biomarker Value",
    coloraxis_colorbar=dict(title="LOS (Days)"), # Single gradient bar
    template="plotly_white"
)

ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'color'

Did you mean "polar"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardle

In [ ]:
fig.show()

# above this 


In [34]:
df['value_as_number'].describe()

count    24326.000000
mean       167.863624
std        644.159564
min          0.000000
25%          1.200000
50%         13.100000
75%         98.000000
max      26876.000000
Name: value_as_number, dtype: float64

In [52]:
import plotly.express as px
import pandas as pd

# 1. FORCE PLOS TO BE NUMERIC (The most critical step)
# This converts any strings or objects into floats so Plotly sees a range of numbers
study_df["plos_days"] = pd.to_numeric(study_df["plos_days"], errors='coerce')

# 2. DROP ROWS WITHOUT LOS DATA
# Missing values can break the continuous color scale
study_df = study_df.dropna(subset=["plos_days"])
print(study_df.describe())

print(study_df['plos_days'].dtype)  # Confirm it's numeric

# 3. GENERATE THE PLOT
fig = px.line(
    study_df.sort_values(by=['index_visit_id', 'days_from_proc']),
    x="days_from_proc", 
    y="value_as_number",
    color="plos_days",                # The scale is based on PLOS
    line_group="index_visit_id",      # Each visit is a separate line
    markers=True,                     # Each point is a measurement
    hover_data=["person_id", "plos_days"],
    title=f"Biomarker Trends by Post-Op LOS"
)

# 4. OVERRIDE TO CONTINUOUS BLUE SCALE
# This removes the multi-colored dots and replaces them with a single Blue gradient bar
fig.update_layout(
    coloraxis_colorscale="Blues_r",  # Dark Blue (Short LOS) to Light Blue (Long LOS)
    template="plotly_white",
    xaxis_title="Days Relative to Procedure (0 = Procedure Day)",
    yaxis_title="Measurement Value",
    coloraxis_colorbar=dict(title="Post-Op LOS (Days)")
)

# 5. MARK THE PROCEDURE EVENT
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Procedure")

fig.show()

         person_id  index_visit_id  procedure_concept_id  \
count  2492.000000     2492.000000                2492.0   
mean   1572.272873     4884.855136             4284104.0   
min     944.000000       39.000000             4284104.0   
25%    1341.250000     2655.000000             4284104.0   
50%    1565.000000     4986.000000             4284104.0   
75%    1872.250000     7210.000000             4284104.0   
max    2088.000000     9286.000000             4284104.0   
std     317.467185     2649.301715                   0.0   

                  procedure_datetime  measurement_concept_id  \
count                           2492                  2492.0   
mean   2023-09-30 19:37:04.719100928              42868738.0   
min              2023-01-11 00:00:00              42868738.0   
25%              2023-05-29 00:00:00              42868738.0   
50%              2023-10-03 00:00:00              42868738.0   
75%              2024-01-29 12:00:00              42868738.0   
max        

KeyError: 'day_from_procedure'

In [54]:
import plotly.express as px
import pandas as pd
import numpy as np

# --- STEP 1: DEFINE TARGETS ---
# Replace these with your actual IDs
target_procedure = 4203153  
target_biomarker = 3034022

# --- STEP 2: FILTER & CLEAN DATA ---
# Filter for the specific study criteria
study_df = df[(df['procedure_concept_id'] == target_procedure) & 
              (df['measurement_concept_id'] == target_biomarker)].copy()

# Force numeric types for the continuous color scale and time axis
study_df["plos_days"] = pd.to_numeric(study_df["plos_days"], errors='coerce')
study_df = study_df.dropna(subset=["plos_days"])

# Sort values so the lines draw from left-to-right correctly
study_df = study_df.sort_values(by=['index_visit_id', 'days_from_proc'])

# --- STEP 3: CONFIGURE COLOR SCALE ---
# Calculate the range for the color bar
los_min = study_df["plos_days"].min()
los_max = study_df["plos_days"].max()

# Use Blues_r for Dark (Short LOS) to Light (Long LOS)
colorscale_name = "Blues_r" 

# --- STEP 4: GENERATE THE PLOT ---
fig = px.line(
    study_df,
    x="days_from_proc", 
    y="value_as_number",
    color="plos_days",                # This triggers the continuous mapping
    line_group="index_visit_id",      # Keeps each visit as a distinct line
    markers=True,                     # Shows individual measurement points
    hover_data=["person_id", "index_visit_id", "plos_days"],
    title=f"Biomarker {target_biomarker} Trends for Procedure {target_procedure}"
)

# --- STEP 5: FORCE CONTINUOUS VISUALS ---
# This step ensures you get a single gradient bar instead of discrete dots
fig.update_layout(
    coloraxis_colorscale=colorscale_name,
    coloraxis_showscale=True,
    template="plotly_white",
    xaxis_title="Days Relative to Procedure (0 = Procedure Day)",
    yaxis_title="Measurement Value",
    coloraxis_colorbar=dict(title="Post-Op LOS (Days)")
)

# Add the clinical baseline vertical line
fig.add_vline(x=0, line_dash="dash", line_color="red", 
              annotation_text="Procedure Event")

# --- STEP 6: DISPLAY ---
fig.show()

In [43]:
import plotly.express as px
import pandas as pd

# Define targets
target_measurement = 3034022  # Creatinine
target_procedure = 4203153    # CABG

# Filter data
plot_df = df[
    (df['measurement_concept_id'] == target_measurement) & 
    (df['procedure_concept_id'] == target_procedure)
].copy()

# Validation checks
print(f"Total measurements: {len(plot_df)}")
print(f"Unique visits: {plot_df['index_visit_id'].nunique()}")
print(f"Unique patients: {plot_df['person_id'].nunique()}")
print(f"LOS range: {plot_df['plos_days'].min():.1f} - {plot_df['plos_days'].max():.1f} days")
print(f"Date range: {plot_df['days_from_proc'].min()} to {plot_df['days_from_proc'].max()} days")

# Check if we have enough data
if len(plot_df) < 10:
    print(f"⚠️ Warning: Only {len(plot_df)} measurements found. Consider different targets.")
else:
    # Ensure numeric types
    plot_df["plos_days"] = pd.to_numeric(plot_df["plos_days"], errors='coerce')
    plot_df = plot_df.dropna(subset=["plos_days", "value_as_number"])
    
    # Sort for proper line rendering
    plot_df = plot_df.sort_values(by=['index_visit_id', 'days_from_proc'])
    
    # Calculate summary stats
    median_los = plot_df.groupby('index_visit_id')['plos_days'].first().median()
    n_visits = plot_df['index_visit_id'].nunique()
    
    # Create visualization
    fig = px.line(
        plot_df,
        x="days_from_proc", 
        y="value_as_number",
        color="plos_days",
        line_group="index_visit_id",
        markers=True,
        color_continuous_scale="Blues_r",
        range_color=[plot_df["plos_days"].min(), plot_df["plos_days"].max()],
        hover_data={
            "person_id": True,
            "plos_days": ":.1f",
            "phase_label": True,
            "days_from_proc": False,
            "value_as_number": ":.2f"
        },
        title=(f"Creatinine Trends for CABG Patients<br>"
               f"<sub>N={n_visits} visits | Median LOS: {median_los:.1f} days</sub>")
    )
    
    # Add reference lines
    fig.add_vline(x=0, line_dash="dash", line_color="red", 
                  annotation_text="CABG Procedure", 
                  annotation_position="top")
    
    # Optional: Add normal creatinine range (adjust as needed)
    fig.add_hrect(y0=0.7, y1=1.3, fillcolor="green", opacity=0.1,
                  annotation_text="Normal Range", annotation_position="top left")
    
    fig.update_layout(
        template="plotly_white",
        xaxis_title="Days Relative to Procedure (0 = Surgery Day)",
        yaxis_title="Creatinine (mg/dL)",
        coloraxis_colorbar=dict(
            title="Post-Op LOS<br>(Days)",
            tickformat=".0f"
        ),
        hovermode="closest",
        width=1200,
        height=700
    )
    
    fig.show()
    
    # Optional: Save the figure
    # fig.write_html(f"creatinine_cabg_trends.html")
    
    # Show distribution of LOS
    print("\n📊 LOS Distribution:")
    print(plot_df.groupby('index_visit_id')['plos_days'].first().describe())

Total measurements: 671
Unique visits: 263
Unique patients: 262
LOS range: 0.0 - 28.0 days
Date range: -428.3885482523148 to 245.9974178587963 days


TypeError: line() got an unexpected keyword argument 'color_continuous_scale'

In [12]:
target_measurement = 3034022  # Example: Creatinine
target_procedure = 4203153      # Example: CABG
plot_df = df[(df['measurement_concept_id'] == target_measurement) & 
             (df['procedure_concept_id'] == target_procedure)].copy()

In [37]:
fig = px.line(
    plot_df,
    x="los_days",          # e.g., 'days_since_procedure'
    y="value_as_number",    # The biomarker value
    color="los_days",             # CONTINUOUS SCALE: Mapping color to Post-Proc LOS
    line_group="index_visit_id", # IDENTITY: Each line is a unique visit
    markers=True,             # POINTS: Shows each distinct measurement
    hover_data=["person_id", "los_days", "value_as_number"], # HOVER: Additional info on hover
    title="Biomarker Measurements Over Post-Procedure Length of Stay",
    labels={
        "los_days": "Post-Procedure Length of Stay (Days)",
        "value_as_number": "Biomarker Value",
        "index_visit_id": "Visit ID"
    }
)

In [38]:
fig.update_layout(
    coloraxis_colorscale='Blues_r',
    template="plotly_white"
)

In [14]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"
fig.show()